In [295]:
# coding: utf-8
import csv
import nltk
import pandas as pd # data processing, CSV file I/O
import numpy as np # linear algebra
import os
import re 
import string
import emoji

from bs4 import BeautifulSoup
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [296]:
# import dataset csv
dataset = pd.read_csv("Gundala6k.csv")

In [297]:
dataset.head()

,No,Datetime,User,Text
0,0,2020-02-17 14:24:36+00:00,BerisikRadio,#interview bareng Compadres gimana akhirnya te...
1,1,2020-02-17 12:02:00+00:00,channelmaenan,GUNDALA ARMORED SUIT (Upgrade) 1:6 statue Limi...
2,2,2020-02-15 14:14:08+00:00,Fow_Lagih,kirain rame.. #gundala
3,3,2020-02-14 12:08:07+00:00,roul_am,Nice combat with martial arts #gundala
4,4,2020-02-14 06:51:18+00:00,bapakranger,@Bumilangit Corner @GandariaCity mall L2 Skuy ...


In [298]:
dataset['Text'] = dataset['Text'].apply(str)

In [299]:
# cari emoticon yang dipake di dataset ini
tweets_text = dataset.Text.str.cat()
emos = set(re.findall(r" ([xX:;][-*']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count, reverse=True)

[(592, ':/'),
 (50, ':('),
 (33, ':)'),
 (12, ":')"),
 (10, ':D'),
 (5, ':3'),
 (3, ':|'),
 (3, ':p'),
 (3, ":'v"),
 (2, ':v'),
 (2, ':o'),
 (1, ":'D"),
 (1, ":'9")]

In [300]:
POS_EMO = r" ([xX;:]'?[dD)]|:'?[\)]|[;:][pP]|[:]'?[v3]) "
NEG_EMO = r" (:'?[\/|\(]) "
print("Happy emoticons:", set(re.findall(POS_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(NEG_EMO, tweets_text)))

Happy emoticons: {':p', ":'v", ':D', ':)', ":'D", ":')", ':3', ':v'}
Sad emoticons: {':(', ':|', ':/'}


In [301]:
dataset = dataset[dataset['Text'] != "nan"]

In [302]:
# PREPROCESSING TEXT : Data Cleaning
def clean_tweets(tweet):    
    # remove link web
    tweet = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})", "", tweet)
    # remove link pic twitter
    tweet = re.sub(r"(pic\.twitter\.com\/[a-zA-Z0-9][a-zA-Z0-9]+)", "", tweet)
    # remove username
    tweet = re.sub(r"@[^\s]+[\s]?", '', tweet)
    # remove 2+ dots with space
    tweet = re.sub(r'\.{2,}', '.', tweet)
    # remove numbers 
    tweet = re.sub(r'\d+', '', tweet)
    # HARUS DILAKUKAN: HAPUS HASHTAG SELAIN #GUNDALA #gUNDALA #Gundala 
    # tweet = re.sub(r'#', '', tweet)
    #tweet = re.sub(r'(?<=#)\w+', '', tweet)
    # remove other characters 
    tweet = re.sub(r'[^ -_:?!.a-z0-9]', '', tweet)
    # replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    # casefolding
    tweet = tweet.lower()
    return tweet

In [303]:
# Apply function preprocessing text to the dataset column [Text]
dataset['Clean Tweets'] = dataset['Text'].apply(clean_tweets)

In [304]:
# SENTENCE SPLITTING
raw_sentences = [sent_tokenize(i) for i in dataset['Clean Tweets']]
dataset['Sentence Splitting'] = raw_sentences

In [305]:
# WORD TOKENIZATION 
def tokenization_w(raw_s): # fungsi untuk tokenisasi
    #w_new = []
    all = [] # buat list baru untuk tampung semuanya 
    for s in raw_s: # untuk semua s di dalam tweet yang sudah di-sentence splitting 
        w_new = [] # buat variable untuk tampung semuanya 
        for w in s: # untuk semua kalimat yang ada di tweet
            w_token = word_tokenize(w) # di tokenize 
            w_new.append(w_token) # 
        all.append(w_new)
    return all

clean_words = tokenization_w(raw_sentences)
dataset['Tweet Tokens'] = clean_words

In [313]:
from nltk.corpus import stopwords
list_stopwords = stopwords.words('indonesian')

In [314]:
# TAMBAH STOPWORDS
list_stopwords.extend(["yg", "dgn", "ny", "kalo", "klo", "kl", "biar", "bikin", 
                       "bilang", "krn", "nya", "nih", "sih","jgn","sdh", "aja",
                      "hehe", "haha", "loh", "ya", "bah"])
list_stopwords.remove("sangat")
list_stopwords.remove("sekali")

In [315]:
# MASUKIN KAMUS KE DATAFRAME
txt_stopwords = pd.read_csv("stopword_list_tala.txt", names=["stopwords"], header=None)

In [316]:
# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopwords["stopwords"][0].split(' '))

In [317]:
# convert list to dictionary 
list_stopwords = set(list_stopwords)
# list_stopwords

In [319]:
# STOPWORDS REMOVER 
def stopwords_w(kalimat):
    all = []
    for item in kalimat:
        temp = []
        for tweet in item:
            if tweet not in list_stopwords:
                temp.append(tweet)
        all.append(temp)
    return all 

dataset['Tweets Stopwords'] = dataset['Tweet Tokens'].apply(stopwords_w)

In [321]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
factory = StopWordRemoverFactory()
stopwords = factory.create_stop_word_remover()
# stemmer = factory.create_stemmer()

In [322]:
# STEMMING 
#def stemmer_w(kalimat):
    #all = []
    #for item in kalimat:
        #temp = []
        #for tweet in item:
            #stem_words = stemmer.stem(tweet) # di tokenize 
            #temp.append(tweet)
        #all.append(temp)
    #return all 

#dataset['Hasil Stemming'] = dataset['Stops'].apply(stemmer_w)

In [323]:
dataset['Sentence Splitting'][338]

['.',
 '#gundala.',
 'walaupun masih banyak kekurangan sana sini tapi sebagai pembuka adiwira indonesia effortnya dan tim patut diacungi jempol']

In [324]:
dataset['Sentence Splitting'][4526]

['aku ga ngerti semua pemainnya bagus.',
 'bahkan yg cuma cameo doang aja aku suka.',
 'nangis pas denger backsoundnya karena sekeren itu #gundala']

In [325]:
dataset['Sentence Splitting'][4349]

['keren banget ini harus nonton lagi dan gak sabar nunggu lanjutannya #gundala #pinkhodie']

In [326]:
dataset['Text'][4526]

'Aku ga ngerti semua pemainnya bagus. Bahkan yg cuma cameo doang aja aku suka. Nangis pas denger backsoundnya karena sekeren itu #Gundala'

In [327]:
dataset['Sentence Splitting'][4526]

['aku ga ngerti semua pemainnya bagus.',
 'bahkan yg cuma cameo doang aja aku suka.',
 'nangis pas denger backsoundnya karena sekeren itu #gundala']

In [328]:
dataset['Tweet Tokens'][4526]

[['aku', 'ga', 'ngerti', 'semua', 'pemainnya', 'bagus', '.'],
 ['bahkan', 'yg', 'cuma', 'cameo', 'doang', 'aja', 'aku', 'suka', '.'],
 ['nangis',
  'pas',
  'denger',
  'backsoundnya',
  'karena',
  'sekeren',
  'itu',
  '#',
  'gundala']]

In [329]:
dataset['Tweets Stopwords'][4526]

[['ga', 'ngerti', 'pemainnya', 'bagus', '.'],
 ['cameo', 'doang', 'suka', '.'],
 ['nangis', 'pas', 'denger', 'backsoundnya', 'sekeren', '#', 'gundala']]

In [330]:
dataset['Tweet Tokens'][2133]

[['#',
  'gundala',
  'sempet',
  'gamau',
  'nonton',
  'tp',
  'akhirnya',
  'jd',
  'nonton',
  'wkwk',
  'dan',
  'apa',
  '?'],
 ['suka', 'bangettt', '!'],
 ['gue',
  'fikir',
  'dih',
  'jd',
  'superhero',
  'pasti',
  'cringe',
  'bgt',
  'tau',
  'nya',
  'ngga',
  'malah',
  'keren']]

In [331]:
dataset['Tweets Stopwords'][2133]

[['#',
  'gundala',
  'sempet',
  'gamau',
  'nonton',
  'tp',
  'jd',
  'nonton',
  'wkwk',
  '?'],
 ['suka', 'bangettt', '!'],
 ['gue',
  'fikir',
  'dih',
  'jd',
  'superhero',
  'cringe',
  'bgt',
  'tau',
  'ngga',
  'keren']]

In [332]:
dataset['Sentence Splitting'][2441]

['akhirnya loh nonton #gundala!!!!!',
 'tidak sempurna tapi epic banget sih warbyasak.',
 'cara memperkenalkan karakter-karakternya juga oke (hint: anak bapak).',
 'gue suka banget jokesnya banyak yang kena juga lawak bat gue antusias banget menyambut kelanjutan bumi langit universe ini.']

In [333]:
dataset['Tweet Tokens'][2441]

[['akhirnya', 'loh', 'nonton', '#', 'gundala', '!', '!', '!', '!', '!'],
 ['tidak', 'sempurna', 'tapi', 'epic', 'banget', 'sih', 'warbyasak', '.'],
 ['cara',
  'memperkenalkan',
  'karakter-karakternya',
  'juga',
  'oke',
  '(',
  'hint',
  ':',
  'anak',
  'bapak',
  ')',
  '.'],
 ['gue',
  'suka',
  'banget',
  'jokesnya',
  'banyak',
  'yang',
  'kena',
  'juga',
  'lawak',
  'bat',
  'gue',
  'antusias',
  'banget',
  'menyambut',
  'kelanjutan',
  'bumi',
  'langit',
  'universe',
  'ini',
  '.']]

In [334]:
dataset['Tweets Stopwords'][2441]

[['nonton', '#', 'gundala', '!', '!', '!', '!', '!'],
 ['sempurna', 'epic', 'banget', 'warbyasak', '.'],
 ['memperkenalkan',
  'karakter-karakternya',
  'oke',
  '(',
  'hint',
  ':',
  'anak',
  ')',
  '.'],
 ['gue',
  'suka',
  'banget',
  'jokesnya',
  'kena',
  'lawak',
  'bat',
  'gue',
  'antusias',
  'banget',
  'menyambut',
  'kelanjutan',
  'bumi',
  'langit',
  'universe',
  '.']]

In [335]:
dataset['Sentence Splitting'][2916]

['#gundala keren!!!!!!',
 'awalan yg bagus buat bumilangit cinematic universe.',
 'gw percaya ini bakal sebooming dan gak kalah keren sama mcu!']

In [336]:
dataset['Tweet Tokens'][2916]

[['#', 'gundala', 'keren', '!', '!', '!', '!', '!', '!'],
 ['awalan', 'yg', 'bagus', 'buat', 'bumilangit', 'cinematic', 'universe', '.'],
 ['gw',
  'percaya',
  'ini',
  'bakal',
  'sebooming',
  'dan',
  'gak',
  'kalah',
  'keren',
  'sama',
  'mcu',
  '!']]

In [337]:
dataset['Text'][2441]

'Akhirnya loh nonton #Gundala!!!!! Tidak sempurna tapi epic banget sih warbyasak. Cara memperkenalkan karakter-karakternya juga oke (hint: anak Bapak). Gue suka banget jokesnya banyak yang kena juga lawak bat Gue antusias banget menyambut kelanjutan Bumi Langit Universe ini. '

In [338]:
dataset['Clean Tweets'][2441]

'akhirnya loh nonton #gundala!!!!! tidak sempurna tapi epic banget sih warbyasak. cara memperkenalkan karakter-karakternya juga oke (hint: anak bapak). gue suka banget jokesnya banyak yang kena juga lawak bat gue antusias banget menyambut kelanjutan bumi langit universe ini. '

In [339]:
dataset['Tweet Tokens'][338]

[['.'],
 ['#', 'gundala', '.'],
 ['walaupun',
  'masih',
  'banyak',
  'kekurangan',
  'sana',
  'sini',
  'tapi',
  'sebagai',
  'pembuka',
  'adiwira',
  'indonesia',
  'effortnya',
  'dan',
  'tim',
  'patut',
  'diacungi',
  'jempol']]

In [340]:
dataset['Tweets Stopwords'][338]

[['.'],
 ['#', 'gundala', '.'],
 ['kekurangan',
  'pembuka',
  'adiwira',
  'indonesia',
  'effortnya',
  'tim',
  'patut',
  'diacungi',
  'jempol']]

In [341]:
dataset['Tweets Stopwords'][5543]

[['overall',
  'sinematografinya',
  'sangat',
  'bagus',
  'tweaks',
  'here',
  'and',
  'there',
  'but',
  'still',
  'i',
  "'m",
  'proud',
  'of',
  'this',
  'movie',
  'and',
  'i',
  'love',
  'it',
  'it',
  "'s",
  'the',
  'first',
  'step',
  '.'],
 ['semoga',
  'menerima',
  'kritik',
  'and',
  'use',
  'them',
  'to',
  'be',
  'even',
  'better',
  'semangatt',
  '!'],
 ['#', 'gundala']]

In [345]:
dataset['Tweets Stopwords'][515]

[['very', 'quotable', '#', 'gundala', 'ni', '.'],
 ['suka',
  'sangat',
  'kata-kata',
  'agung',
  'ga',
  'gunanya',
  'hidup',
  'ga',
  'peduli',
  'mikirin',
  '!'],
 ['menusuk', 'jiwa', '.']]

In [355]:
dataset['Hasil Stemming'][338]

[[''],
 ['', 'gundala', ''],
 ['kurang',
  'buka',
  'adiwira',
  'indonesia',
  'effortnya',
  'tim',
  'patut',
  'acung',
  'jempol']]

In [349]:
# STEMMING 
def stemmer_w(kalimat):
    all = []
    for item in kalimat:
        temp = []
        for tweet in item:
            stem_words = stemmer.stem(tweet) # di tokenize 
            temp.append(stem_words)
        all.append(temp)
    return all 

dataset['Hasil Stemming'] = dataset['Tweets Stopwords'].apply(stemmer_w)

In [354]:
dataset['Hasil Stemming'][2735]

[['gokil', 'gundala', 'bagus', 'bangetttt', '', ''],
 ['koreo',
  'antem',
  'keren',
  'banget',
  'adegan',
  'sadis',
  'ngeri',
  'main',
  'manteb',
  'tp',
  'adegan',
  'tbtb',
  'ubah',
  'jd',
  'film',
  'horror',
  'wkwk',
  '',
  'gundala']]

In [357]:
dataset['Hasil Stemming'][2133]

[['',
  'gundala',
  'sempet',
  'gamau',
  'nonton',
  'tp',
  'jd',
  'nonton',
  'wkwk',
  ''],
 ['suka', 'bangettt', ''],
 ['gue',
  'fikir',
  'dih',
  'jd',
  'superhero',
  'cringe',
  'bgt',
  'tau',
  'ngga',
  'keren']]

In [358]:
dataset['Hasil Stemming'][2441]

[['nonton', '', 'gundala', '', '', '', '', ''],
 ['sempurna', 'epic', 'banget', 'warbyasak', ''],
 ['kenal', 'karakter', 'oke', '', 'hint', '', 'anak', '', ''],
 ['gue',
  'suka',
  'banget',
  'jokesnya',
  'kena',
  'lawak',
  'bat',
  'gue',
  'antusias',
  'banget',
  'sambut',
  'lanjut',
  'bumi',
  'langit',
  'universe',
  '']]